In [90]:
%%capture

%pip install python-dotenv requests

In [91]:
from dotenv import load_dotenv
load_dotenv()

True

In [113]:
import os

slack_token = os.getenv('SLACK_BOT_TOKEN')
slack_channel = '#chatops-tests'
slack_icon_emoji = None # ':see_no_evil:'
slack_user_name = None # "Eve 2"

In [93]:
import requests

def get_channel_id(channel_name):
    response = requests.get(
        'https://slack.com/api/conversations.list',
        headers={'Authorization': f'Bearer {slack_token}'},
        params={'exclude_archived': 'true', 'limit': 1000}
    ).json()
    
    if not response.get('ok'):
        raise Exception(f"Failed to fetch channels: {response.get('error')}")

    for channel in response.get('channels', []):
        if f"#{channel['name']}" == channel_name:
            return channel['id']
    
    raise Exception(f"Channel '{channel_name}' not found")

slack_channel_id = get_channel_id(slack_channel)

In [114]:
import json
import requests

def post_message_to_slack(text, blocks = None, unfurl_links = True):
    return requests.post('https://slack.com/api/chat.postMessage', {
        'token': slack_token,
        'channel': slack_channel,
        'text': text,
        'blocks': json.dumps(blocks) if blocks else None,
        'unfurl_links': unfurl_links,
        'icon_emoji': slack_icon_emoji,
        'username': slack_user_name,
    }).json()

In [115]:
wiki_link = "https://en.wikipedia.org/wiki/Three_Laws_of_Robotics#:~:text=A%20robot%20may%20not%20injure%20a%20human%20being%20or%2C%20through%20inaction%2C%20allow%20a%20human%20being%20to%20come%20to%20harm."
slack_info = 'A robot *may not injure a human being* or, through inaction, allow a human being to come to harm. <{}|First law of Robotics>.'.format(wiki_link)

response = post_message_to_slack(slack_info, unfurl_links = False)

In [96]:
blocks = [
  {
    "type":"section",
    "text": { "type":"mrkdwn", "text":":robot_face: The *Three Laws of Robotics* (often shortened to The Three Laws or Asimov's Laws) are a set of rules devised by science fiction author *Isaac Asimov*, which were to be followed by robots in several of his stories. The laws are:" }
  },
  {
    "type":"rich_text",
    "elements": [
      {
        "type":"rich_text_list",
        "style":"ordered",
        "indent":0,
        "elements": [
          {
            "type":"rich_text_section",
            "elements": [{ "type":"text", "text":"A robot may not injure a human being or, through inaction, allow a human being to come to harm." }]
          },
          {
            "type":"rich_text_section",
            "elements": [{ "type":"text", "text":"A robot must obey the orders given it by human beings except where such orders would conflict with the First Law."} ]
          },
          {
            "type":"rich_text_section",
            "elements": [{ "type":"text", "text":"A robot must protect its own existence as long as such protection does not conflict with the First or Second Law." }]
          }
        ]
      }
    ]
  },
  {
    "type":"context",
    "elements": [{ "type":"mrkdwn", "text":":books: Source: <https://en.wikipedia.org/wiki/Three_Laws_of_Robotics|Three Laws of Robotics (Wikipedia)>." }]
  }
]

slack_info = 'Here are the Three Laws of Robotics.'
response = post_message_to_slack(slack_info, blocks, unfurl_links = False)

In [97]:
import requests

def post_file_to_slack(text, file_name, file_bytes, snippet_type=None, title=None):

    # Step 1: Get upload URL and file ID
    upload_url_data = requests.post('https://slack.com/api/files.getUploadURLExternal', {
        'token': slack_token,
        'filename': file_name,
        'length': len(file_bytes),
        'snippet_type': snippet_type
    }).json()
    
    if not upload_url_data.get('ok'):
        raise Exception(f"Failed to get upload URL: {upload_url_data.get('error')}")
    
    upload_url = upload_url_data['upload_url']
    file_id = upload_url_data['file_id']

    # Step 2: Upload the file to the provided URL
    requests.post(
        upload_url,
        files  = { 'file': file_bytes },
    )

    # Step 3: Complete the file upload
    complete_upload_data = requests.post('https://slack.com/api/files.completeUploadExternal',
        headers={
            'Authorization': f'Bearer {slack_token}',
            'Content-Type': 'application/json; charset=utf-8'
        },
        json={
            'files': [{ 'id': file_id, 'title': title or file_name }],
            'channel_id': slack_channel_id,
            'initial_comment': text,
    }).json()
    
    if not complete_upload_data.get('ok'):
        raise Exception(f"Failed to complete file upload: {complete_upload_data.get('error')}")
    
    return complete_upload_data

In [98]:
import urllib.request

# download image
image = "https://townsquare.media/site/442/files/2018/06/wall-e-eve.jpg"
response = urllib.request.urlopen(image)
data = response.read()

# send to slack
response = post_file_to_slack(
  ':palm_tree: Amazing day with *WALL-E*. Check out this photo! :sparkles:',
  'wall-e-and-eve-wallpaper-2.jpg',
  data,
  title="Day at the beach!")